In [1]:
import os

os.chdir("../")
%pwd

'c:\\Users\\jlrodriguez.andreu\\dev\\lab-mlops\\mlops-ml-project'

In [2]:
# entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path


In [3]:
# config manager
from src.mlproject.constants import *
from src.mlproject.utils.common import read_yaml, create_directories


In [9]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_transformation_config(self) -> DataTransformationConfig:

        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )

        return data_transformation_config



In [10]:
# component
import os
from src.mlproject import logger
from sklearn.model_selection import train_test_split
import pandas as pd


In [11]:
class DataTransformation:

    def __init__(self, config: DataTransformationConfig):
        self.config = config

    ## you can add different data transformation techniques as scaler, pca
    # aqui solo hacemos train test split, pero seria el sitio donde añadir las transformaciones

    def train_test_splitting(self):

        data = pd.read_csv(self.config.data_path)

        train, test = train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("splitted data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)


In [12]:
# pipeline
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_splitting()
except Exception as e:
    raise e


[2024-06-11 17:19:11,903: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-11 17:19:11,904: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-11 17:19:11,906: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-11 17:19:11,907: INFO: common: created directory at: artifacts]
[2024-06-11 17:19:11,908: INFO: common: created directory at: artifacts/data_transformation]
[2024-06-11 17:19:11,935: INFO: 4155829181: splitted data into training and test sets]
[2024-06-11 17:19:11,936: INFO: 4155829181: (1199, 12)]
[2024-06-11 17:19:11,937: INFO: 4155829181: (400, 12)]
(1199, 12)
(400, 12)
